### **Transportation Networks**

In [2]:
from pyomo.environ import *

### Background

The prototypical transportation problem deals with the distribution of a commodity from a set of sources to a set of destinations. The object is to minimize total transportation costs while satisfying constraints on the supplies available at each of the sources, and satisfying demand requirements at each of the destinations.

Here we illustrate the transportation problem using an example from Chapter 5 of Johannes Bisschop, "AIMMS Optimization Modeling", Paragon Decision Sciences, 1999. In this example there are two factories and six customer sites located in 8 European cities as shown in the following map. The customer sites are labeled in red, the factories are labeled in blue.

![TransportationNetworksMap.png](https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/figures/TransportationNetworksMap.png?raw=1)

Transportation costs between sources and destinations are given in units of &euro;/ton of goods shipped, and list in the following table along with source capacity and demand requirements.


#### Table of transportation costs, customer demand, and available supplies

| Customer\Source | Arnhem [&euro;/ton] | Gouda [&euro;/ton] | Demand [tons]|
| :--: | :----: | :---: | :----: |
| London | - | 2.5 | 125 |
| Berlin | 2.5 | - | 175 |
| Maastricht | 1.6 | 2.0 | 225 |
| Amsterdam | 1.4 | 1.0 | 250 |
| Utrecht | 0.8 | 1.0 | 225 |
| The Hague | 1.4 | 0.8 | 200 |
| **Supply [tons]** | 550 tons | 700 tons |  |

The situation can be modeled by links connecting a set nodes representing sources to a set of nodes representing customers.

![TransportNet.png](https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/figures/TransportNet.png?raw=1)

For each link we can have a parameter $T[c,s]$ denoting the cost of shipping a ton of goods over the link. What we need to determine is the amount of goods to be shipped over each link, which we will represent as a non-negative decision variable $x[c,s]$.

The problem objective is to minimize the total shipping cost to all customers from all sources. 

$$minimize:\quad Cost = \sum_{c \in Customers}\sum_{s \in Sources} T[c,s] x[c,s]$$

Shipments from all sources can not exceed the manufacturing capacity of the source.

$$\sum_{c \in Customers} x[c,s] \leq Supply[s] \qquad \forall s \in Sources$$

Shipments to each customer must satisfy their demand.

$$\sum_{s\in Sources} x[c,s] = Demand[c] \qquad \forall c \in Customers$$

In [3]:
Demand = {
   'Lon': 125,        # London
   'Ber': 175,        # Berlin
   'Maa': 225,        # Maastricht
   'Ams': 250,        # Amsterdam
   'Utr': 225,        # Utrecht
   'Hag': 200         # The Hague
}

Supply = {
   'Arn': 550,        # Arnhem
   'Gou': 700         # Gouda
}

T = {
    ('Lon', 'Arn'): 1000, # Big numbers to discourage choice
    ('Lon', 'Gou'): 2.5,
    ('Ber', 'Arn'): 2.5,
    ('Ber', 'Gou'): 1000,
    ('Maa', 'Arn'): 1.6,
    ('Maa', 'Gou'): 2.0,
    ('Ams', 'Arn'): 1.4,
    ('Ams', 'Gou'): 1.0,
    ('Utr', 'Arn'): 0.8,
    ('Utr', 'Gou'): 1.0,
    ('Hag', 'Arn'): 1.4,
    ('Hag', 'Gou'): 0.8
}

In [4]:
# Step 0: Create an instance of the model
model = ConcreteModel()

# Step 1: Define index sets
CUS = list(Demand.keys())
SRC = list(Supply.keys())

# Step 2: Define the decision 
model.x = Var(CUS, SRC, domain = NonNegativeReals)

# Step 3: Define Objective
@model.Objective(sense=minimize)
def cost(m):
    return sum([T[c,s]*model.x[c,s] for c in CUS for s in SRC])

# Step 4: Constraints
@model.Constraint(SRC)
def src(m, s):
    return sum([model.x[c,s] for c in CUS]) <= Supply[s]

@model.Constraint(CUS)
def dmd(m, c):
    return sum([model.x[c,s] for s in SRC]) == Demand[c]
    
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1715.0
  Upper bound: 1715.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.050310373306274414
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [5]:
model.x.display()

x : Size=12, Index=x_index
    Key            : Lower : Value : Upper : Fixed : Stale : Domain
    ('Ams', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Ams', 'Gou') :     0 : 250.0 :  None : False : False : NonNegativeReals
    ('Ber', 'Arn') :     0 : 175.0 :  None : False : False : NonNegativeReals
    ('Ber', 'Gou') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Hag', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Hag', 'Gou') :     0 : 200.0 :  None : False : False : NonNegativeReals
    ('Lon', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Lon', 'Gou') :     0 : 125.0 :  None : False : False : NonNegativeReals
    ('Maa', 'Arn') :     0 : 225.0 :  None : False : False : NonNegativeReals
    ('Maa', 'Gou') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Utr', 'Arn') :     0 : 150.0 :  None : False : False : NonNegativeReals
    ('Utr', 'Gou') :     0 :  75.0 :  None : Fa

In [6]:
for c in CUS:
    for s in SRC:
        print(c, s, model.x[c,s]())

Lon Arn 0.0
Lon Gou 125.0
Ber Arn 175.0
Ber Gou 0.0
Maa Arn 225.0
Maa Gou 0.0
Ams Arn 0.0
Ams Gou 250.0
Utr Arn 150.0
Utr Gou 75.0
Hag Arn 0.0
Hag Gou 200.0


In [7]:
if 'ok' == str(results.Solver.status):
    print("Total Shipping Costs = ", model.cost())
    print("\nShipping Table:")
    for s in SRC:
        for c in CUS:
            if model.x[c,s]() > 0:
                print("Ship from ", s," to ", c, ":", model.x[c,s]())
else:
    print("No Valid Solution Found")

Total Shipping Costs =  1715.0

Shipping Table:
Ship from  Arn  to  Ber : 175.0
Ship from  Arn  to  Maa : 225.0
Ship from  Arn  to  Utr : 150.0
Ship from  Gou  to  Lon : 125.0
Ship from  Gou  to  Ams : 250.0
Ship from  Gou  to  Utr : 75.0
Ship from  Gou  to  Hag : 200.0
